In [12]:
import tensorflow as tf
from IPython.display import display
import pandas as pd
import seaborn as sns
import tensorflow.keras.backend as K
import numpy as np
import os
import gc
import glob
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import BatchNormalization, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard, ReduceLROnPlateau
from sklearn.utils import class_weight
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
%matplotlib inline

In [2]:
def reset_graph(model=None):
    if model:
        try:
            del model
        except:
            return False
    
    tf.reset_default_graph()
    
    K.clear_session()
    
    gc.collect()
    
    return True


# reset callbacks 
def reset_callbacks(checkpoint=None, reduce_lr=None, early_stopping=None, tensorboard=None):
    checkpoint = None
    reduce_lr = None
    early_stopping = None
    tensorboard = None

In [3]:
reset_graph()
reset_callbacks()

In [4]:
image_ext = '.jpeg'
data_dir = 'dataset/'
test_dir = data_dir+'test/'
train_dir = data_dir+'train/'
val_dir = data_dir+'val/'
normals = 'NORMAL/'
infected = 'PNEUMONIA/'
training_normal_nos = len(glob.glob1(train_dir+normals,'*'+image_ext))
training_infected_nos = len(glob.glob1(train_dir+infected,'*'+image_ext))
training_nos = training_infected_nos+training_normal_nos
testing_normal_nos = len(glob.glob1(test_dir+normals,'*'+image_ext))
testing_infected_nos = len(glob.glob1(test_dir+infected,'*'+image_ext))
testing_nos = testing_infected_nos+testing_normal_nos
validation_normal_nos = len(glob.glob1(val_dir+normals,'*'+image_ext))
validation_infected_nos = len(glob.glob1(val_dir+infected,'*'+image_ext))
validation_nos = validation_infected_nos+validation_normal_nos

print('Total number of training images: ', training_nos)
print('Total number of testing images: ', testing_nos)
print('Total number of validation images: ', validation_nos)

Total number of training images:  5216
Total number of testing images:  624
Total number of validation images:  16


In [5]:
rescale = 1./255
target_size = (150, 150)
batch_size = 163
class_mode = "categorical"
# class_mode = "binary"


train_datagen = ImageDataGenerator(
    rescale=rescale,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)


train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=batch_size,
    shuffle=True)


validation_datagen = ImageDataGenerator(rescale=rescale)

validation_generator = validation_datagen.flow_from_directory(
    val_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=validation_nos,
    shuffle = False)


test_datagen = ImageDataGenerator(rescale=rescale)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=target_size,
    class_mode=class_mode,
    batch_size=testing_nos,
    shuffle = False)

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [6]:
from sklearn.utils import class_weight
def get_weight(y):
    class_weight_current =  class_weight.compute_class_weight('balanced', np.unique(y), y)
    return class_weight_current

In [7]:
class_weight = get_weight(train_generator.classes)
class_weight

array([1.9448173 , 0.67303226])

In [8]:
reset_graph()
reset_callbacks()

In [19]:
base = 'bin'
model_dir = os.path.join(base,'models')
log_dir = os.path.join(base,'logs')
model_file = model_dir + "{epoch:02d}-val_acc-{val_acc:.2f}-val_loss-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(
    model_file, 
    monitor='val_acc', 
    save_best_only=True)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    verbose=1,
    restore_best_weights=True)


tensorboard = TensorBoard(
    log_dir=log_dir,
    batch_size=batch_size,
    update_freq = 'batch')


reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=5,
    cooldown=2,
    min_lr=0.0000000001,
    verbose=1)


#-----------------------------------------------------------------------------------------------------------------#
callbacks = [checkpoint, reduce_lr, early_stopping, tensorboard]

In [13]:
def get_model():    
    
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
    x = base_model.output    
    x = BatchNormalization()(x)
    x = Flatten()(x)
    predictions = Dense(2, activation='softmax')(x) 
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    model.summary()        
    return model

In [14]:
model = get_model()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 150, 150, 3) 0                                            
__________________________________________________________________________________________________
conv2d_94 (Conv2D)              (None, 74, 74, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, 74, 74, 32)   96          conv2d_94[0][0]                  
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 74, 74, 32)   0           batch_normalization_95[0][0]     
____________________________________________________________________________________________

In [15]:
steps_per_epoch=len(train_generator)
validation_steps=len(validation_generator)
optimizer=Adam()
loss='categorical_crossentropy'
metrics=['accuracy']
epochs = 2
model.compile(optimizer, loss=loss, metrics=metrics)

In [20]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = steps_per_epoch,
    epochs=epochs,
    verbose=2,
    callbacks=callbacks,
    validation_data=validation_generator,
    validation_steps=validation_steps, 
    class_weight=class_weight)

Epoch 1/2
32/32 - 227s - loss: 0.5197 - acc: 0.8691 - val_loss: 4.9152 - val_acc: 0.5625
Epoch 2/2


KeyboardInterrupt: 